In [2]:
import os, pickle
import gensim, logging
from gensim.models import Word2Vec, Phrases, phrases, KeyedVectors
import scipy, numpy


logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)

In [3]:
data_output_folder = '../../../Data/output'
entity_rep_output_folder = 'Entity_Ranking/EntityRep'
entity_context_word_file = 'doc_entity_context_word.p'


word_emnbedding_pretrained_trained_on_corpus = '../../../word_embeddings/trained_word_embeddings/word2vec/word_pretrain_trained_on_corpus/w2v_pretain_corpus_trained_gensim_300.txt'

In [4]:
# Load the pickle file
doc_role_entity_context_word_dict = pickle.load(open(os.path.join(data_output_folder, entity_rep_output_folder, entity_context_word_file), 'rb'))

In [5]:
word_vectors = KeyedVectors.load_word2vec_format(word_emnbedding_pretrained_trained_on_corpus, binary=False)

2018-04-29 04:55:03,657 : INFO : loading projection weights from ../../../word_embeddings/trained_word_embeddings/word2vec/word_pretrain_trained_on_corpus/w2v_pretain_corpus_trained_gensim_300.txt
2018-04-29 04:57:19,463 : INFO : loaded (410894, 300) matrix from ../../../word_embeddings/trained_word_embeddings/word2vec/word_pretrain_trained_on_corpus/w2v_pretain_corpus_trained_gensim_300.txt


In [6]:
word_vector = word_vectors.get_vector('Narendra')
size = len(word_vector)
#print(type(word_vector))
word_vectors.distance('Jammu', 'Narendra')

0.84234518702434402

In [7]:
doc_role_entity_context_word_centroid_dict = dict()
for doc in doc_role_entity_context_word_dict.keys():
    tag_dict = doc_role_entity_context_word_dict[doc]
    tag_centroid_dict = dict()
    for tag in tag_dict.keys():
        entity_list = tag_dict[tag]
        entity_centroid_list = list()
        for entity in entity_list:
            num_of_words_in_entity = len(entity)
            entity_centroid = numpy.zeros(size)
            for word in entity:
                entity_centroid = numpy.add(entity_centroid, word_vectors.get_vector(word))
            entity_centroid = entity_centroid/num_of_words_in_entity
            entity_centroid_list.append(entity_centroid)
        tag_centroid_dict[tag] = entity_centroid_list
    doc_role_entity_context_word_centroid_dict[doc] = tag_centroid_dict

In [8]:
#for key in doc_role_entity_context_word_centroid_dict.keys():
#    print(key, '\n')
#    print(doc_role_entity_context_word_centroid_dict[key], '\n')
    
pickle.dump(doc_role_entity_context_word_centroid_dict, open('../../../Data/output/Entity_Ranking/EntityRep/doc_role_entity_context_word_centroid.p', 'wb'))